<center>
<h3> Project 3 - Scientific Computing </h3>
<hr>
  
*Letizia D'Achille*
</center>

### Report of numerical results

In [ ]:
# Libraries and global variables

import matplotlib.pyplot as plt
import pandas as pd
from time import time
from project3 import *

tol = 1e-8

##### Multigrid in 1d

**Problem:**  

Let $\Omega = (0,1)$.  Given $f\in C^0(\overline\Omega)$ find a $u\in C^2(\overline\Omega)$ such that

\begin{align*}
	−u'' + (2-|x|)u & =  (2x-1)^3 && \quad\text{in } \Omega \\
	u & = 0 && \quad\text{on } \delta \Omega.\\
\end{align*}

**Remark**: We start from an initial guess $\hat u_h^{(0)} = 0$, therefore the residuals at the beginning of the iteration are those reported in the table that follows. We will see how these residuals will decrease using the different methods.

In [ ]:
# Residuals computation - 1d

d = {"l" : [], "n" : [], "res" : []}

for l in range(3,15):
	n = 2**l
	h = 1/n
	x = np.linspace(0,1,n+1)
	c = 2 - abs(x)
	uh = np.zeros_like(x)
	fh = (2*x-1)**3

	rh = uh.copy()
	rh[1:n] = fh[1:n] + (uh[0:n-1] + uh[2:n+1] - (2 + c[1:n] * h**2) * uh[1:n]) / h**2
	r = la.norm(rh, np.inf)

	d["l"].append(l)
	d["n"].append(2**l)
	d["res"].append(r)

df = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 6})
df_1 = df.iloc[:6:]
df_2 = df.iloc[6::]
df_1.index = [i for i in range(6)]
df_2.index = [i for i in range(6)]
pd.concat([df_1,emptycol,df_2],axis = 1).style \
	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

The following table is related to the weighted Jacobi method.

In [ ]:
# Execution of weighted Jacobi method - 1d

d = {"l" : [], "n" : [], "ω" : [], "it" : [], "res" : [], "t" : []}

for l in range(3,10):
	for omega in [1/3, 2/3]:
		n = 2**l
		h = 1/n
		x = np.linspace(0,1,n+1)
		c = 2 - abs(x)
		uh = np.zeros_like(x)
		fh = (2*x-1)**3

		t = time()
		k = 0
		s = 1 + tol
		while s > tol:
			s = jacobi_step_1d(uh, fh, omega)
			k += 1
		t = time() - t

		rh = uh.copy()
		rh[1:n] = fh[1:n] + (uh[0:n-1] + uh[2:n+1] - (2 + c[1:n] * h**2) * uh[1:n]) / h**2
		r = la.norm(rh, np.inf)

		d["l"].append(l)
		d["n"].append(2**l)
		if abs(omega - 1/3) < 0.1: 
			d["ω"].append("1/3") 
		else: 
			d["ω"].append("2/3")
		d["it"].append(k)
		d["res"].append(r)
		d["t"].append(t)

df_j1d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 7})
df_j1d_1 = df_j1d.iloc[::2]
df_j1d_2 = df_j1d.iloc[1::2]
df_j1d_1.index = [i for i in range(7)]
df_j1d_2.index = [i for i in range(7)]
pd.concat([df_j1d_1,emptycol,df_j1d_2],axis = 1).style \
	.set_caption("Weighted Jacobi method - 1d") \
	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

We can immediately observe that all the collected measures grow as $n$ grows. In particular, for a fixed value of $\omega$, we can look at the rate of convergence. Note that:

$$\rho(C_{J_\omega}) = 1 - 2\omega\sin^2\left(h\frac{\pi}{2}\right) = 1 - 2\omega\left(h^2\frac{\pi^2}{4} + O(h^4)\right) = 1 - \frac{\omega}{2}h^2\pi^2 + O(h^4)$$

This estimation does not truly reflects the convergence rate because of the smoothing property of the weighted Jacobi method, but we can still note that the number of iterations are about halved when $\omega$ is doubled. However, we can see that the rate of convergence for both $\omega = \frac{1}{3},\frac{2}{3}$ is about of the order of $h^\frac{3}{2}$, where $h = \frac{1}{n}$. Therefore, the ratio between the number of iterations for $\omega = \frac{1}{3}$ and the one for $\omega = \frac{2}{3}$ tend to diminish as $n$ grows.  
For fixed $n$, the timings are proportional to the number of iterations, therefore we obtain that case $\omega = \frac{2}{3}$ is about twice as fast as case $\omega = \frac{1}{3}$.

Collecting the residuals, we notice that these also grow as $n$ increases. This behavior, which can also be observed in the subsequent tests, can be explained by the fact that a smaller mesh size equals a larger number of points on which we have to approximate the solution. Thus, in general, there is a greater probability that with the same tolerance on the pseudo-residual we will have a solution that struggles to correctly approximate all points at once.  
We can observe that also the residuals are halved when $\omega$ is doubled. In this case the ratio seems to stay the same for all values of $n$. Therefore for $\omega = \frac{2}{3}$ we obtain a more accurate solution in less time than for $\omega = \frac{1}{3}$. We can conclude that over-smoothing is not beneficial to the rate of convergence in the cases under consideration, even if asymptotically the discrepancy could be narrowed and this trend could be reversed.

The following table is related to the two-grid correction scheme.

In [ ]:
# Execution of two-grid correction scheme - 1d

d = {"l" : [], "n" : [], "ω" : [], "it" : [], "res" : [], "t" : []}

for l in range(3,10):
	for omega in [1/3, 2/3]:
		n = 2**l
		h = 1/n
		x = np.linspace(0,1,n+1)
		c = 2 - abs(x)
		uh = np.zeros_like(x)
		fh = (2*x-1)**3

		t = time()
		k = 0
		s = 1 + tol
		while s > tol:
			s = two_grid_correction_step_1d(uh, fh, omega)
			k += 1
		t = time() - t

		rh = uh.copy()
		rh[1:n] = fh[1:n] + (uh[0:n-1] + uh[2:n+1] - (2 + c[1:n] * h**2) * uh[1:n]) / h**2
		r = la.norm(rh, np.inf)

		d["l"].append(l)
		d["n"].append(2**l)
		if abs(omega - 1/3) < 0.1: 
			d["ω"].append("1/3") 
		else: 
			d["ω"].append("2/3")
		d["it"].append(k)
		d["res"].append(r)
		d["t"].append(t)

df_t1d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 7})
df_t1d_1 = df_t1d.iloc[::2]
df_t1d_2 = df_t1d.iloc[1::2]
df_t1d_1.index = [i for i in range(7)]
df_t1d_2.index = [i for i in range(7)]
pd.concat([df_t1d_1,emptycol,df_t1d_2],axis = 1).style \
	.set_caption("Two-grid correction scheme - 1d") \
  	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

The considerations made for the weighted Jacobi method still hold for this scheme. Indeed the latter uses the former as building block.  

We now want to compare these two methods. It is clear why iterations decrease, in fact a single iteration of the new scheme includes multiple iterations of the weighted Jacobi method. Moreover we can observe that the ratio between the needed iterations for the base method and those for the new scheme is greater than the number of steps of Jacobi used in the scheme $(12)$. This shows that the reduction to the coarsed grid is much more effective than the simple reiteration of $12$ steps of Jacobi.  
Since the total number of iterations of weighted Jacobi method is indeed reduced, we consequently have that the time required to reach the required tolerance is smaller for the new method. 
Moreover, we can notice that, for fixed $\omega$ and two fixed values of $n$, while the ratio between two values of `it` is the same as before, the ratio between the collected timings is reduced. This happens because we are redirecting the computation of one step of Jacobi on a coarsed grid, that is less expensive on a time basis.

We can notice that for $n$ and $\omega$ fixed, also the residuals associated to the solution obtained at the end of the algorithm are smaller than in the previous case, therefore we can conclude that this scheme is actually improving the previous one in the cases taken into consideration.

The following table is related to the W-cycle method.

In [ ]:
# Execution of W-cycle method - 1d

d = {"l" : [], "n" : [], "ω" : [], "α1, α2":[], "it" : [], "res" : [], "t" : []}

for l in range(3,15):
	for alpha1 in [1, 2]:
		for omega in [1/3, 2/3]:
			alpha2 = alpha1
			
			n = 2**l
			h = 1/n
			x = np.linspace(0,1,n+1)
			c = 2 - abs(x)
			uh = np.zeros_like(x)
			fh = (2*x-1)**3

			t = time()
			k = 0
			s = 1 + tol
			while s > tol:
				s = w_cycle_step_1d(uh, fh, omega, alpha1, alpha2)
				k += 1
			t = time() - t
			
			rh = uh.copy()
			rh[1:n] = fh[1:n] + (uh[0:n-1] + uh[2:n+1] - (2 + c[1:n] * h**2) * uh[1:n]) / h**2
			r = la.norm(rh, np.inf)

			d["l"].append(l)
			d["n"].append(2**l)
			if abs(omega - 1/3) < 0.1: 
				d["ω"].append("1/3") 
			else: 
				d["ω"].append("2/3")
			d["α1, α2"].append(alpha1)
			d["it"].append(k)
			d["res"].append(r)
			d["t"].append(t)

df_w1d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 24})
df_w1d_1 = df_w1d.iloc[::2]
df_w1d_2 = df_w1d.iloc[1::2]
df_w1d_1.index = [i for i in range(24)]
df_w1d_2.index = [i for i in range(24)]
pd.concat([df_w1d_1,emptycol,df_w1d_2],axis = 1).style \
	.set_caption("W-cycle - 1d") \
	.format({'res': "{:.5e}"}) \
  	.hide(axis='index')

First, let us analyse the pairs of observations for $\omega$ and $n$ fixed. It is clear that when $\alpha_1$ and $\alpha_2$ are greater, the total number of steps required decrease, since a single step includes a greater number of iterations of the base method.  
However, we cannot find any general rule about whether is better to increase $\alpha_1$ and $\alpha_2$ or not, since both the timings and the residuals have no fixed order relation. Indeed, there are cases when we can reach the tolerance with fewer steps when $\alpha_1=\alpha_2=2$, then the timings result better. However, when we reach the tolerance with the same number of steps for both the possible values, then the case with $\alpha_1=\alpha_2=1$ has an advantage since it executes a fewer number of operations. For example, we see this trend when $n$ becomes high, and the tolerance is always reached with one single step.  

On the other hand, if we fix $\alpha_1$, $\alpha_2$ and $n$, one can observe that if the tolerance is reached with the same number of iterations for the two values of $\omega$, we can see the timings are the same. This is quite obvious since one step of the W-cycle method takes the same amount of time for both values of $\omega$. This becomes relevant for higher values of $n$ where the tolerance is reached with $1/2$ iterations in both cases. In those cases we can look at the residuals and observe that we obtain smaller values for $\omega = \frac{2}{3}$, which is then again the best choice.  
Even when we do not have a match between the number of iterations, we can observe that $\omega = \frac{2}{3}$ is better since it always requires a smaller number of steps. Also the residuals mantain the same order relation.  

We immediately observe that, unlike the previous methods, the number of iterations does not increase (indeed, it decreases), which means they do not depend on $n$ as expected from the theory.  
Also the timings have a different behaviour with respect to the previous tables: again they adhere to theory, as the complexity appears to be $O(n)$.


The following table is related to the full multigrid method.

In [ ]:
# Execution of full multigrid method - 1d

d = {"l" : [], "n" : [], "α1, α2":[], "ν": [], "p-res" : [], "res" : [], "t" : []}

for l in range(3,15):
	for alpha1, nu in [(1,1),(1,2),(2,1),(2,2)]:
		omega = 2/3
		alpha2 = alpha1
		
		n = 2**l
		h = 1/n
		x = np.linspace(0,1,n+1)
		c = 2 - abs(x)
		uh = np.zeros_like(x)
		fh = (2*x-1)**3

		t = time()
		s = full_mg_1d(uh, fh, omega, alpha1, alpha2, nu)
		t = time() - t

		rh = uh.copy()
		rh[1:n] = fh[1:n] + (uh[0:n-1] + uh[2:n+1] - (2 + c[1:n] * h**2) * uh[1:n]) / h**2
		r = la.norm(rh, np.inf)

		d["l"].append(l)
		d["n"].append(2**l)
		d["α1, α2"].append(alpha1)
		d["ν"].append(nu)
		d["p-res"].append(s)
		d["res"].append(r)
		d["t"].append(t)

df_f1d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 24})
df_f1d_1 = df_f1d.iloc[::2]
df_f1d_2 = df_f1d.iloc[1::2]
df_f1d_1.index = [i for i in range(24)]
df_f1d_2.index = [i for i in range(24)]
pd.concat([df_f1d_1,emptycol,df_f1d_2],axis = 1).style \
	.set_caption("Full multigrid - 1d (ω = 2/3)") \
	.format({'res': "{:.5e}", 'p-res': "{:.5e}"}) \
	.hide(axis='index')

Let us analyse the observations with respect to the value of $\nu$. We know that changing the value of $\nu$ only modifies the number of steps of the W-cycle method at the end of the execution of the method. Then it becomes clear why the timings for $\nu = 2$ are twice as high as those for $\nu = 1$.  
For the same reason both the residuals and pseudo-residuals decrease for an higher $\nu$. In particular, the pseudo-residuals decrease by one order of magnitude, while the residuals, especially for the higher values of $l$, are squared. This a great advantage, since the we gain a quadratic reduction on residuals while we have a linear increase on timings. This happens because we are spending more time on the coarser grids, and for high $l$ this becomes convenient.  
Moreover, the considerations about the different values of $\alpha_1$ and $\alpha_2$ made for the W-cycle method can be repeated in a similar manner. Since we are executing a single step, we will clearly have worst timings for $\alpha_1=\alpha_2=2$, but in this case we reach better residuals.

As $n$ grows, we can notice that pseudo-residuals decrease. This can be explained looking at how one step of the W-cycle method works. Indeed, this executes a great number of iterations of Jacobi on different grids in order to reach the coarser one. In particular, it executes an high number of corrections, one for every grid it goes through (this number corresponds to $l$). Hence at the end, even if it is more expensive in terms of time, we reach a smaller pseudo-residual. Here is also why, in the table related to W-cycle method, we observe a reduction of the number of iterations as $n$ grows. In this table we can indeed see that, for the higher values of $l$, the tolerance required in the previous cases is easily reached by a single step of the W-cycle. Additionally, we can note that we have a few more cases that immediately reach the required tolerance in one step than in a straightforward implementation of the W-cycle. This is due to the first phase of the full multigrid method, in which we obtain an approximation from which we can start the execution of the W-cycle.  

On the other hand, as $n$ grows, the residuals slightly increase. As we already remarked, this is an expectable behaviour since we work on increasingly fine grids. 

##### Approximation to the solution - 1d

In [ ]:
# Computation of an approximation to the solution of 1d problem and its minimum (and maximum), and plot

l = 14; omega = 2/3;  alpha2 = alpha1 = 2; nu = 2

n = 2**l
h = 1/n
x = np.linspace(0,1,n+1)
uh = np.zeros_like(x)
fh = fh = (2*x-1)**3

_ = full_mg_1d(uh, fh, omega, alpha1, alpha2, nu)

argmin = np.argmin(uh)
argmax = np.argmax(uh)
print("-"*50)
print(f"min = {uh[argmin]}")
print(f"max = {uh[argmax]}")
print("-"*50)

fig, ax = plt.subplots(1)
ax.plot(x, uh)
ax.plot(x[argmin], uh[argmin], c='k', marker='.')
ax.plot([x[argmin],x[argmin]], [0,uh[argmin]], c='k', ls = '--', lw = '1')
ax.plot([0,x[argmin]], [uh[argmin],uh[argmin]], c='k', ls = '--', lw = '1')
ax.spines['left'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['top'].set_color('none')
plt.show()

In order to approximate the solution, we first chose $l = 14$ to obtain a figure as homogeneous as possible. Among the two methods for which we computed the solution for this value of $l$, and among the various parameters, we then chose the ones for which we obtained the smaller residual. Hence, we chose full-multigrid method with parameters $\alpha1 = \alpha2 = 2$, $\nu = 2$. The minimum was computed using the final solution, and it was plotted together with the approximation.

##### Multigrid in 2d

**Problem:**  

Let $\Omega = (0,1)^2$.  Given $f\in C^0(\overline\Omega)$ find a $u\in C^2(\overline\Omega)$ such that

\begin{align*}
	−\Delta u + \left(2-\sqrt{x_1^2 + x_2^2}\ \right)u & =  3 x_1^4 - x_2^2 && \quad\text{in } \Omega \\
	u & = 0 && \quad\text{on } \delta \Omega.\\
\end{align*}

**Remark**: Again we start from an initial guess $\hat u_h^{(0)} = 0$, therefore the residuals at the beginning of the iteration are those reported in the table that follows. 

In [ ]:
# Residuals computation - 2d

d = {"l" : [], "n" : [], "res" : []}

for l in range(2,12):
	n = 2**l
	h = 1/n
	x = np.linspace(0, 1, n+1)
	y = np.linspace(0, 1, n+1)
	x, y = np.meshgrid(x, y)
	c = 2 - np.sqrt(x**2 + y**2)
	uh = np.zeros_like(x)
	fh = 3*x**4 - y**2

	rh = uh.copy()
	rh[1:n,1:n] = fh[1:n,1:n] + (uh[0:n-1,1:n] + uh[2:n+1,1:n] + uh[1:n,0:n-1] + uh[1:n,2:n+1] - (4 + c[1:n,1:n] * h**2) * uh[1:n,1:n]) / h**2
	r = la.norm(rh.flat, np.inf)

	d["l"].append(l)
	d["n"].append(2**l)
	d["res"].append(r)

df = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 5})
df_1 = df.iloc[:5:]
df_2 = df.iloc[5::]
df_1.index = [i for i in range(5)]
df_2.index = [i for i in range(5)]
pd.concat([df_1,emptycol,df_2],axis = 1).style \
	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

The following table is related to the weighted Jacobi method.

In [ ]:
# Execution of weighted Jacobi method - 2d

d = {"l" : [], "n" : [], "ω" : [], "it" : [], "res" : [], "t" : []}

for l in range(2,8):
	for omega in [1/3, 2/3]:
		n = 2**l
		h = 1/n
		x = np.linspace(0, 1, n+1)
		y = np.linspace(0, 1, n+1)
		x, y = np.meshgrid(x, y)
		c = 2 - np.sqrt(x**2 + y**2)
		uh = np.zeros_like(x)
		fh = 3*x**4 - y**2

		t = time()
		k = 0
		s = 1 + tol
		while s > tol:
			s = jacobi_step_2d(uh, fh, omega)
			k += 1
		t = time() - t

		rh = uh.copy()
		rh[1:n,1:n] = fh[1:n,1:n] + (uh[0:n-1,1:n] + uh[2:n+1,1:n] + uh[1:n,0:n-1] + uh[1:n,2:n+1] - (4 + c[1:n,1:n] * h**2) * uh[1:n,1:n]) / h**2
		r = la.norm(rh.flat, np.inf)
		
		d["l"].append(l)
		d["n"].append(2**l)
		if abs(omega - 1/3) < 0.1: 
			d["ω"].append("1/3") 
		else: 
			d["ω"].append("2/3")
		d["it"].append(k)
		d["res"].append(r)
		d["t"].append(t)

df_j2d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 6})
df_j2d_1 = df_j2d.iloc[::2]
df_j2d_2 = df_j2d.iloc[1::2]
df_j2d_1.index = [i for i in range(6)]
df_j2d_2.index = [i for i in range(6)]
pd.concat([df_j2d_1,emptycol,df_j2d_2],axis = 1).style \
	.set_caption("Weighted Jacobi method - 2d") \
	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

All the remarks made for 1d case can be brought to this case. Indeed,

$$C_{J_\omega}^{2d} = I - \left(\frac{1}{\omega}D\right)^{-1}A_h = I - \frac{\omega}{4}h^2A_h$$

$$\rho(C_{J_\omega}^{2d}) = 1 - 2\omega\sin^2\left(h\frac{\pi}{2}\right) = \rho(C_{J_\omega})$$

We can notice that in this case the timings grow faster than in 1d with respect to the iterations. Indeed the number of operation per step is squared than in the previous case as the number of points on which we are approximating the solution is squared, therefore we could expect this behaviour. 
The values for the number of iterations themselves are higher than in 1d, but this mainly depends on the different types of problem we are taking into consideration.

The following table is related to the two-grid correction scheme.

In [ ]:
# Execution of two-grid correction scheme - 2d

d = {"l" : [], "n" : [], "ω" : [], "it" : [], "res" : [], "t" : []}

for l in range(2,8):
	for omega in [1/3, 2/3]:
		n = 2**l
		h = 1/n
		x = np.linspace(0, 1, n+1)
		y = np.linspace(0, 1, n+1)
		x, y = np.meshgrid(x, y)
		c = 2 - np.sqrt(x**2 + y**2)
		uh = np.zeros_like(x)
		fh = 3*x**4 - y**2

		t = time()
		k = 0
		s = 1 + tol
		while s > tol:
			s = two_grid_correction_step_2d(uh, fh, omega)
			k += 1
		t = time() - t

		rh = uh.copy()
		rh[1:n,1:n] = fh[1:n,1:n] + (uh[0:n-1,1:n] + uh[2:n+1,1:n] + uh[1:n,0:n-1] + uh[1:n,2:n+1] - (4 + c[1:n,1:n] * h**2) * uh[1:n,1:n]) / h**2
		r = la.norm(rh.flat, np.inf)

		d["l"].append(l)
		d["n"].append(2**l)
		if abs(omega - 1/3) < 0.1: 
			d["ω"].append("1/3") 
		else: 
			d["ω"].append("2/3")
		d["it"].append(k)
		d["res"].append(r)
		d["t"].append(t)

df_t2d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 6})
df_t2d_1 = df_t2d.iloc[::2]
df_t2d_2 = df_t2d.iloc[1::2]
df_t2d_1.index = [i for i in range(6)]
df_t2d_2.index = [i for i in range(6)]
pd.concat([df_t2d_1,emptycol,df_t2d_2],axis = 1).style \
	.set_caption("Two-grid correction scheme - 2d") \
  	.format({'res': "{:.5e}"}) \
	.hide(axis='index')

Again, the considerations remain unchanged. Note that in 2d we are starting from a smaller $n$, and for the smallest values of this parameter behaviour seems dissimilar. However, even if we have few observations, we can see that when $n$ becomes big the method performs in the same way.  
The improvement in timings are now much more convenient than in 1d, since these tend to grow very quickly.

The following table is related to the W-cycle method.

In [ ]:
# Execution of W-cycle method - 2d

d = {"l" : [], "n" : [], "ω" : [], "α1, α2":[], "it" : [], "res" : [], "t" : []}

for l in range(2,11):
	for alpha1 in [1, 2]:
		for omega in [1/3, 2/3]:
			alpha2 = alpha1
			
			n = 2**l
			h = 1/n
			x = np.linspace(0, 1, n+1)
			y = np.linspace(0, 1, n+1)
			x, y = np.meshgrid(x, y)
			c = 2 - np.sqrt(x**2 + y**2)
			uh = np.zeros_like(x)
			fh = 3*x**4 - y**2

			t = time()
			k = 0
			s = 1 + tol
			while s > tol:
				s = w_cycle_step_2d(uh, fh, omega, alpha1, alpha2)
				k += 1
			t = time() - t

			rh = uh.copy()
			rh[1:n,1:n] = fh[1:n,1:n] + (uh[0:n-1,1:n] + uh[2:n+1,1:n] + uh[1:n,0:n-1] + uh[1:n,2:n+1] - (4 + c[1:n,1:n] * h**2) * uh[1:n,1:n]) / h**2
			r = la.norm(rh.flat, np.inf)

			d["l"].append(l)
			d["n"].append(2**l)
			if abs(omega - 1/3) < 0.1: 
				d["ω"].append("1/3") 
			else: 
				d["ω"].append("2/3")
			d["α1, α2"].append(alpha1)
			d["it"].append(k)
			d["res"].append(r)
			d["t"].append(t)

df_w2d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 18})
df_w2d_1 = df_w2d.iloc[::2]
df_w2d_2 = df_w2d.iloc[1::2]
df_w2d_1.index = [i for i in range(18)]
df_w2d_2.index = [i for i in range(18)]
pd.concat([df_w2d_1,emptycol,df_w2d_2],axis = 1).style \
	.set_caption("W-cycle - 2d") \
	.format({'res': "{:.5e}"}) \
  	.hide(axis='index')

In this case, since we do not reach a case in which for different values of $\alpha_1$, $\alpha_2$ we have the same number of iterations, we can state that the case $\alpha_1 = \alpha_2 = 2$ appears to be better than the other. Indeed, both the timings and the residuals seem to experience a non-negligible improvement. 
We only have a couple of counterexamples for $l=7,8$ and $\omega = \frac{2}{3}$.

We can still have the same remarks as in the case 1d, but in this case the number of unknowns is squared, therefore the timings should follow the theoretical complexity $O(n^2)$. In practice, we still have a linear growth, at least for the values considered for $n$. This probably is caused by the efficient implementation of operations among numpy arrays, that do not make the computation explode for $n$ this small. We begin to see the quadratic behavious in the step from $l=9$ to $l=10$.

The following table is related to the full multigrid method.

In [ ]:
# Execution of full multigrid method - 2d

d = {"l" : [], "n" : [], "α1, α2":[], "ν": [], "p-res" : [], "res" : [], "t" : []}

for l in range(2,11):
	for alpha1, nu in [(1,1),(1,2),(2,1),(2,2)]:
		omega = 2/3
		alpha2 = alpha1

		n = 2**l
		h = 1/n
		x = np.linspace(0, 1, n+1)
		y = np.linspace(0, 1, n+1)
		x, y = np.meshgrid(x, y)
		c = 2 - np.sqrt(x**2 + y**2)
		uh = np.zeros_like(x)
		fh = 3*x**4 - y**2

		t = time()
		s = full_mg_2d(uh, fh, omega, alpha1, alpha2, nu)
		t = time() - t

		rh = uh.copy()
		rh[1:n,1:n] = fh[1:n,1:n] + (uh[0:n-1,1:n] + uh[2:n+1,1:n] + uh[1:n,0:n-1] + uh[1:n,2:n+1] - (4 + c[1:n,1:n] * h**2) * uh[1:n,1:n]) / h**2
		r = la.norm(rh.flat, np.inf)

		d["l"].append(l)
		d["n"].append(2**l)
		d["α1, α2"].append(alpha1)
		d["ν"].append(nu)
		d["p-res"].append(s)
		d["res"].append(r)
		d["t"].append(t)

df_f2d = pd.DataFrame(data = d)
emptycol = pd.DataFrame(data = {" " : [' '] * 18})
df_f2d_1 = df_f2d.iloc[::2]
df_f2d_2 = df_f2d.iloc[1::2]
df_f2d_1.index = [i for i in range(18)]
df_f2d_2.index = [i for i in range(18)]
pd.concat([df_f2d_1,emptycol,df_f2d_2],axis = 1).style \
	.set_caption("Full multigrid - 2d (ω = 2/3)") \
	.format({'res': "{:.5e}", 'p-res': "{:.5e}"}) \
	.hide(axis='index')

The remarks made in the previous sections can be analogously repeated here. In 2d the residuals are not squared when doubling $\nu$, but there is still a great improvement with respect to the modifications in timings.  
Note that the residuals are always high with respect to those obtained in 1d case, but we have to consider that the starting error was high as well. We still have the good property of slow growth of residuals.

##### Approximation to the solution - 2d

In [ ]:
# Computation of an approximation to the solution of 2d problem and its minimum (and maximum), and plot

l = 10; omega = 2/3; alpha2 = alpha1 = 2

n = 2**l
h = 1/n
x = np.linspace(0, 1, n+1)
y = np.linspace(0, 1, n+1)
x, y = np.meshgrid(x, y)
uh = np.zeros_like(x)
fh = 3*x**4 - y**2

s = 1 + tol
while  s > tol:
	s = w_cycle_step_2d(uh, fh, omega, alpha1, alpha2)

argmin = np.unravel_index(np.argmin(uh), uh.shape)
argmax = np.unravel_index(np.argmax(uh), uh.shape)
print("-"*50)
print(f"min = {uh[argmin]}")
print(f"max = {uh[argmax]}")
print("-"*50)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(x, y, uh)
plt.plot(x[argmin], y[argmin], uh[argmin], c='k', marker='.')
ax.view_init(10, -140)
plt.show()

In order to approximate the solution, we first chose $l = 10$ to obtain a figure as homogeneous as possible. Among the two methods for which we computed the solution for this value of $l$, and among the various parameters, we then chose the ones for which we obtained the smaller residual, as we did for 1d. In this case, we chose W-cycle method with parameters $\omega = \frac{2}{3}$ and $\alpha1 = \alpha2 = 2$. The minimum was computed using the final solution, and it was plotted together with the approximation.  
Looking at the plot also becomes clear how this problem is not comparable to the one in 1d, indeed the solution is further from the initial approximation than the one in 1d, and behaves quite differently. In particular, this also clearly confirms the calculation of initial residuals.